# SPEED UP MODEL TRAINING
### 본 문서는 PyTorch Lightning의 [공식 가이드](https://pytorch-lightning.readthedocs.io/en/latest/guides/speed.html)의 한글 번역본입니다. (옮긴이 [dnap512](https://github.com/dnap512))

모델의 수렴 시간을 단축할 수 있는 여러 가지 방법이 있습니다.

- gpu/tpu training
- mixed precision (16-bit) training
- control training epochs
- control validation frequency
- limit dataset size
- preload data into ram
- model toggling
- set grads to none
- things to avoid



## GPU/TPU training

**사용 조건:** 이 옵션은 언제나 사용 가능합니다!

Lightning을 사용하면 GPU, TPU 또는 여러 노드에서 실행하는 것을 플래그의 전환으로 간단히 사용할 수 있습니다.

### GPU training

Lightning은 분산 GPU 훈련 속도를 더욱 높일 수 있는 다양한 플러그인을 지원합니다. 주목할만한 것들은 다음과 같습니다.

- [DDPPlugin](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.plugins.training_type.DDPPlugin.html#pytorch_lightning.plugins.training_type.DDPPlugin)
- [DDPShardedPlugin](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.plugins.training_type.DDPShardedPlugin.html#pytorch_lightning.plugins.training_type.DDPShardedPlugin)
- [DeepSpeedPlugin](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.plugins.training_type.DeepSpeedPlugin.html#pytorch_lightning.plugins.training_type.DeepSpeedPlugin)


In [ ]:
# run on 1 gpu
trainer = Trainer(gpus=1)

# train on 8 gpus, using DDP plugin
trainer = Trainer(gpus=8, accelerator="ddp")

# train on multiple GPUs across nodes (uses 8 gpus in total)
trainer = Trainer(gpus=2, num_nodes=4)

#### GPU 훈련 가속 팁

단일 또는 다중 GPU 머신에서 모델을 훈련할 때 Lightning은 처리량, 메모리 효율성 및 모델 Scaling을 향상하기 위한 다수의 고급 optimization 기법들을 제공합니다. 자세한 내용은 [고급 GPU 최적화 훈련](https://pytorch-lightning.readthedocs.io/en/latest/advanced/advanced_gpu.html)을 참조하십시오.

#### Prefer DDP over DP

[`DataParallelPlugin`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.plugins.training_type.DataParallelPlugin.html#pytorch_lightning.plugins.training_type.DataParallelPlugin)은 모든 배치에 대해 세 가지 GPU 전송을 수행합니다.

1. Copy model to device.
2. Copy data to device.
3. Copy outputs of each device back to master.

반면 [`DDPPlugin`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.plugins.training_type.DDPPlugin.html#pytorch_lightning.plugins.training_type.DDPPlugin)은 sync gradients에 대해 1번의 전송만 수행하므로 DDP가 DP보다 훨씬 빠릅니다.

#### DDP를 사용할 때 설정할 것: find_unused_parameters=False

기본적으로 우리는 과거에 발생한 호환성 문제 때문에 find_unused_parameters를 True로 설정했습니다(자세한 내용은 [discussion](https://github.com/PyTorchLightning/pytorch-lightning/discussions/6219) 참조). 이것은 기본적으로 성능이 저하되며, 대부분의 경우 비활성화할 수 있습니다.

In [ ]:
from pytorch_lightning.plugins import DDPPlugin

trainer = pl.Trainer(
    gpus=2,
    plugins=DDPPlugin(find_unused_parameters=False),
)

#### Multi-node 클러스터에서 DDP를 사용하는 경우, NCCL 매개변수 설정

[NCCL](https://developer.nvidia.com/nccl)은 PyTorch에서 노드와 GPU 간의 통신을 처리하는 데 사용하는 NVIDIA Collective Communications Library입니다. 이 문제에서 볼 수 있듯이 NCCL 매개변수를 조정할 때 속도 향상 측면에서 이점이 보고되었습니다. 이 문제에서 Transformer XLM-RoBERTa를 훈련할 때 속도가 30% 향상되고 Detectron2로 훈련할 때 15% 개선된 것을 볼 수 있습니다.

NCCL 매개변수는 환경 변수를 통해 조정할 수 있습니다.

> NOTE: AWS와 GCP는 이미 클러스터에 기본값을 설정했습니다. 이는 일반적으로 custom 클러스터 설정에 유용합니다.

- [NCCL_NSOCKS_PERTHREAD](https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/env.html#nccl-nsocks-perthread)
- [NCCL_SOCKET_NTHREADS](https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/env.html#nccl-socket-nthreads)
- [NCCL_MIN_NCHANNELS](https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/env.html#nccl-min-nchannels)

```bash
export NCCL_NSOCKS_PERTHREAD=4
export NCCL_SOCKET_NTHREADS=2
```

#### Dataloaders

DataLoader를 생성할 때 `num_workers > 0`, 그리고 `pin_memory=True`(GPU에만 해당)를 설정합니다.



In [ ]:
Dataloader(dataset, num_workers=8, pin_memory=True)

#### Num_workers

num_workers를 구체적으로 얼마나 많이 지정할지는 까다롭습니다. 다음은 몇 가지 참고 문헌 \[[1](https://discuss.pytorch.org/t/guidelines-for-assigning-num-workers-to-dataloader/813)\] 및 제안 사항에 대한 요약입니다.

1. `num_workers=0`은 주 프로세스만 일괄 처리를 로드함을 의미합니다(병목 현상이 발생할 수 있음).
2. `num_workers=1`은 하나의 작업자(메인 프로세스가 아님)만 데이터를 로드하지만 여전히 느립니다.
3. `num_workers`는 배치 크기와 시스템에 따라 다릅니다.
4. 일반적으로 시작하는 위치는 `num_workers`를 해당 시스템의 CPU 코어 수와 동일하게 설정하는 것입니다. Python의 `os.cpu_count()`를 사용하여 CPU 코어 수를 얻을 수 있지만 배치 크기에 따라 RAM 메모리가 오버플로될 수 있습니다.

> <span style="color:red">경고</span>: `num_workers`를 증가시키면 CPU 메모리 소비도 증가합니다.

가장 좋은 방법은 num_workers를 천천히 늘리고 훈련 속도가 더 이상 향상되지 않으면 중지하는 것입니다.

#### Spawn

`accelerator=ddp_spawn`을 사용하거나 TPU에서 훈련할 때, 여러 GPU/TPU 코어가 사용되는 방식은 내부에서 `.spawn()`을 호출하는 것입니다. 문제는 `.spawn()`을 사용할 때 PyTorch에 `num_workers > 0`을 함께 사용하면 문제가 생깁니다. 따라서 `num_workers`를 늘릴 수 있도록 `accelerator=ddp`를 사용하는 것이 좋습니다. 그러나 스크립트는 다음과 같이 호출 가능해야 합니다.

```bash
python my_program.py
```

### TPU 훈련

여러분은 trainer flag에서 `tpu_cores`를 1 또는 8로 설정할 수 있습니다.

In [ ]:
# train on 1 TPU core
trainer = Trainer(tpu_cores=1)

# train on 8 TPU cores
trainer = Trainer(tpu_cores=8)

8 코어 이상을 사용하기 위해서는, xla_dist 스크립트를 사용하여 이 스크립트를 제출하세요.

예:
```bash
python -m torch_xla.distributed.xla_dist
--tpu=$TPU_POD_NAME
--conda-env=torch-xla-nightly
--env=XLA_USE_BF16=1
-- python your_trainer_file.py
```

더 자세한 사항은 [Accelerators](https://pytorch-lightning.readthedocs.io/en/latest/extensions/accelerators.html#accelerators)와 [Plugins](https://pytorch-lightning.readthedocs.io/en/latest/extensions/plugins.html#plugins) 가이드를 읽어보세요.

---

## Mixed precision (16-bit) 훈련

**사용 조건:**

- GPU에서 메모리 사용을 최적화할 때
- 16비트 정밀도(NVIDIA 파스칼 아키텍처 이상)를 지원하는 GPU가 있을 때
- 최적화 알고리즘(training_step)이 수치적으로 안정적일 때
- 랩에서 쿨한 사람이 되고 싶을 때 :p

[동영상](https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/pl_docs/yt/Trainer+flags+9+-+precision_1.mp4)을 참고하세요!

Mixed precision은 32비트 및 16비트 부동 소수점 사용을 결합하여 모델 훈련 중 메모리 공간을 줄여 성능을 개선하고 최신 GPU에서 +3배의 속도 향상을 달성합니다.

Lightning은 GPU 및 TPU에 대해 Mixed precision 또는 16비트 훈련을 제공합니다.

In [ ]:
# 16-bit precision
trainer = Trainer(precision=16, gpus=4)

---

## 훈련 Epochs 조정

**사용 조건:** 하이퍼파라미터 서치를 실행하여 좋은 초기 매개변수를 찾고 시간, 비용(돈) 또는 전력(환경)을 절약하려는 경우, 이를 통해 비용 효율성을 높이고 동시에 더 많은 실험을 실행할 수 있습니다.

여러분은 Trainer flag를 사용하여 최소 epoch 수에 대한 훈련을 강제하거나 최대 epoch 수로 제한할 수 있습니다. `min_epochs` 및 `max_epochs`를 사용하여 실행할 에포크 수를 설정합니다.

In [ ]:
# DEFAULT
trainer = Trainer(min_epochs=1, max_epochs=1000)

만약 Iteration based training, 즉 Infinite/iterable dataloader를 실행하는 경우 `min_steps` 및 `max_steps` 플래그로 스탭 수를 제어할 수도 있습니다.

In [ ]:
trainer = Trainer(max_steps=1000)

trainer = Trainer(min_steps=100)

여러분은 훈련 시간에 따라 훈련을 중단할 수도 있습니다.

In [ ]:
# Stop after 12 hours of training or when reaching 10 epochs (string)
trainer = Trainer(max_time="00:12:00:00", max_epochs=10)

# Stop after 1 day and 5 hours (dict)
trainer = Trainer(max_time={"days": 1, "hours": 5})

더 자세한것은 [Trainer flags](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-flags)를 참고하세요!

---


## Control Validation Frequency

### Epochs마다 Validation 수행

**사용 조건:** 데이터셋이 작고, Valid를 적게 수행하려는 경우 사용하세요

`check_val_every_n_epoch Trainer` 플래그를 사용하여 매 epoch마다 Valid를 수행하도록 제한할 수 있습니다.

In [ ]:
# DEFAULT
trainer = Trainer(check_val_every_n_epoch=1)

#### 1 훈련 Epoch 내에서 Valid 빈도 설정

**사용 조건:** Training set이 굉장히 크고, 1 Epoch 안에서 Valid를 수행하려고 할 때

대규모 데이터셋의 경우 훈련 루프 내에서 여러 번 유효성 검사를 확인하는 것이 바람직합니다. 1 Epoch 내에서 자주 확인하기 위해 float 수치를 입력하세요. 배치 갯수를 기준으로 하려면 int 타입 k를 입력합니다. `IterableDataset`을 사용하는 경우 int를 사용해야 합니다.

In [ ]:
# DEFAULT
trainer = Trainer(val_check_interval=0.95)

# check every .25 of an epoch
trainer = Trainer(val_check_interval=0.25)

# check every 100 train batches (ie: for `IterableDatasets` or fixed frequency)
trainer = Trainer(val_check_interval=100)

더 자세한 사항은 [Trainer flags](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-flags)를 읽어보세요!

---


## 데이터셋 크기 제한

### 훈련, 검증, 테스트셋의 Subset을 사용하기

**사용 조건:** 디버깅할 때 또는 거대한 데이터셋을 사용할 때

만약 여러분이 데이터셋의 100%를 사용하기 원하지 않는다면 다음 flags를 사용해보세요

In [ ]:
# DEFAULT
trainer = Trainer(
    limit_train_batches=1.0,
    limit_val_batches=1.0,
    limit_test_batches=1.0
)

# check 10%, 20%, 30% only, respectively for training, validation and test set
trainer = Trainer(
    limit_train_batches=0.1,
    limit_val_batches=0.2,
    limit_test_batches=0.3
)

만약 여러분이 Dataloader에서 `shuffle=True`를 사용했다면, 매 Epoch마다 다르게 랜덤한 Subset을 사용할 것입니다. 그렇지 않다면 모든 Epoch동안 같은 Subset을 사용합니다.

> <span style="color:blue">NOTE</span>: `limit_train_batches`, `limit_val_batches` 및 `limit_test_batches`는 `overfit_batches` > 0인 경우 `overfit_batches`에 의해 덮어쓰여집니다. `fast_dev_run=True`이면 `limit_val_batches`가 무시됩니다.
    
> <span style="color:blue">NOTE</span>: `limit_val_batches=0`으로 설정하면 Valid가 비활성화됩니다.
    
더 자세한 사항은 [Trainer flags](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-flags)를 읽어보세요!

---


## RAM에 데이터 사전 로드

**사용 조건:** 한 번에 데이터셋의 모든 샘플에 액세스해야만 할 때

훈련 또는 Preprocessing에서 전체 데이터셋에 대해 많은 작업을 수행해야 할 때, 충분한 공간이 있다면 모든 데이터를 RAM에 저장하는 것이 유리할 수 있습니다. 그러나 훈련 스크립트의 시작 부분에서 모든 데이터를 로드하면 시간이 오래 걸릴 수 있어 개발 프로세스가 느려지는 단점이 있습니다. 또 다른 단점은 Multiprocessing(예: DDP)에서 데이터가 각 프로세스에서 복사된다는 것입니다. 데이터를 RAM에 미리 복사하여 이러한 문제를 극복할 수 있습니다. 대부분의 UNIX 기반 운영 체제는 일반적으로 `/dev/shm`이라는 마운트 지점을 통해 tmpfs에 대한 직접 액세스를 제공합니다.

0. 필요한 경우 공유 메모리를 늘립니다(OS 설명서를 참조하세요).
1. 훈련 데이터를 공유 메모리에 복사합니다.
```bash
cp -r /path/to/data/on/disk /dev/shm/
```
2. 스크립트 또는 커맨드라인 파라미터에 새 데이터 루트를 참조합니다.
```python
datamodule = MyDataModule(data_root="/dev/shm/my_data")
```

---

## 모델 Toggling

**사용 조건:** Distributed 세팅에서 여러 옵티마이저로 그래디언트 누적을 수행할 때

- 현재 옵티마이저를 A로, 다른 모든 옵티마이저를 B로 간주합니다.
- 토글은 B에서 A까지의 모든 매개변수가 False로 설정된 require_grad 속성을 가짐을 의미합니다.
- 컨텍스트 관리자를 종료하면 원래 상태가 복원됩니다.

기울기 누적을 수행할 때 누적 단계에서 기울기 동기화를 수행할 필요가 없습니다. `sync_grad`를 `False`로 설정하면 이 동기화가 차단되고 훈련 속도가 향상됩니다.

`LightningOptimizer`는 고급 사용자를 위해 `toggle_model()` 함수를 [`contextlib.contextmanager()`](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager)로 제공합니다.

다음은 고급 사용 사례의 예입니다.


In [ ]:
# Scenario for a GAN with gradient accumulation every 2 batches and optimized for multiple gpus.
class SimpleGAN(LightningModule):

    def __init__(self):
        super().__init__()
        self.automatic_optimization = False

    def training_step(self, batch, batch_idx):
        # Implementation follows the PyTorch tutorial:
        # https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
        g_opt, d_opt = self.optimizers()

        X, _ = batch
        X.requires_grad = True
        batch_size = X.shape[0]

        real_label = torch.ones((batch_size, 1), device=self.device)
        fake_label = torch.zeros((batch_size, 1), device=self.device)

        # Sync and clear gradients
        # at the end of accumulation or
        # at the end of an epoch.
        is_last_batch_to_accumulate = \
            (batch_idx + 1) % 2 == 0 or self.trainer.is_last_batch

        g_X = self.sample_G(batch_size)

        ##########################
        # Optimize Discriminator #
        ##########################
        with d_opt.toggle_model(sync_grad=is_last_batch_to_accumulate):
            d_x = self.D(X)
            errD_real = self.criterion(d_x, real_label)

            d_z = self.D(g_X.detach())
            errD_fake = self.criterion(d_z, fake_label)

            errD = (errD_real + errD_fake)

            self.manual_backward(errD)
            if is_last_batch_to_accumulate:
                d_opt.step()
                d_opt.zero_grad()

        ######################
        # Optimize Generator #
        ######################
        with g_opt.toggle_model(sync_grad=is_last_batch_to_accumulate):
            d_z = self.D(g_X)
            errG = self.criterion(d_z, real_label)

            self.manual_backward(errG)
            if is_last_batch_to_accumulate:
                g_opt.step()
                g_opt.zero_grad()

        self.log_dict({'g_loss': errG, 'd_loss': errD}, prog_bar=True)

---

## zero_grad 설정

성능을 약간 향상시키기 위해 `optimizer_zero_grad()`를 재정의할 수 있습니다.

이 기술의 장단점에 대한 자세한 설명은 PyTorch 팀이 작성한 [이 문서](https://pytorch.org/docs/master/optim.html#torch.optim.Optimizer.zero_grad)를 참조하세요.

In [ ]:
class Model(LightningModule):

    def optimizer_zero_grad(self, epoch, batch_idx, optimizer, optimizer_idx):
        optimizer.zero_grad(set_to_none=True)

---

## 피해야 할 것들

### .item(), .numpy(), .cpu()

코드 어디에서나 `.item()`을 사용하지 마세요. 연결된 그래프 호출을 제거하려면 대신 `.detach()`를 사용하세요. Lightning은 이를 최적화하기 위해 많은 주의를 기울입니다.

---

### empty_cache()

필요 없을때 이걸 사용하지 마세요! 호출할 때마다 모든 GPU는 동기화를 기다려야 합니다.

---

### 텐서들을 장치로 옮기기

LightningModules는 어떤 장치에 있는지 알고 있습니다! CPU->장치 전송을 피하기 위해 장치에서 직접 텐서를 구성하십시오.

In [ ]:
# bad
t = torch.rand(2, 2).cuda()

# good (self is LightningModule)
t = torch.rand(2, 2, device=self.device)

model attributes가 필요한 텐서의 경우 모듈의 `__init__` 메서드에서 버퍼로 등록하는 것이 가장 좋습니다.

In [ ]:
# bad
self.t = torch.rand(2, 2, device=self.device)

# good
self.register_buffer("t", torch.rand(2, 2))